# 第9章 符合Python风格的对象
得益于Python的数据类型, 自定义类型可以像内置类型那样自然。实现如此自然的行为, 靠的不是继承, 而是鸭子类型(duck typing)

## 9.1 对象表示形式
Python提供了两种表示形式:repr(), str()

## 9.2 再谈向量类

In [20]:
from array import array
import math
# 自己设计向量类v0
class Vector2d(object):
    typecode = "d"
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)  # 实现了iter就会自动拆包
    
    def __str__(self):
        return str(tuple(self)) # 实现了iter方法
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(self)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(self.x or self.y)

In [21]:
v1 = Vector2d(3, 4)
print(v1)  # 调用str方法

(3.0, 4.0)


In [22]:
x, y = v1
x, y  # 拆包

(3.0, 4.0)

In [23]:
 v1  # 调用repr

Vector2d(3.0, 4.0)

In [24]:
v1_clone = eval(repr(v1))
v1_clone

Vector2d(3.0, 4.0)

In [25]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [26]:
abs(v1)

5.0

In [27]:
bool(v1)

True

## 9.3 备选构造方法
使用frombytes方法来实现反序列化

In [50]:
from array import array
import math
# 自己设计向量类v1
class Vector2d(object):
    typecode = "d"
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)  # 实现了iter就会自动拆包
    
    def __str__(self):
        return str(tuple(self)) # 实现了iter方法
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(self)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(self.x or self.y)
    
    @classmethod
    def frombytes(cls, octets):
        cls.typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(cls.typecode)
        return cls(*memv)

In [51]:
v = Vector2d.frombytes(bytes(v1))
v

Vector2d(3.0, 4.0)

## 9.4 classmethod与staticmethod
后者就是定义在类中的普通方法

## 9.5 格式化显示
内置的format()函数和str.format()方法把各个类型的格式化方式委托给相应的\_\_format\_\_(format\_spec)

In [45]:
brl = 1/2.43
brl

0.4115226337448559

In [46]:
format(brl, "0.4f")

'0.4115'

In [47]:
"1 BRL = {rate:0.2f} USD".format(rate=brl)

'1 BRL = 0.41 USD'

In [52]:
from datetime import datetime

In [54]:
now = datetime.now()
now

datetime.datetime(2020, 8, 30, 10, 24, 26, 261079)

In [55]:
format(now, "%H:%M:%S")

'10:24:26'

如果类中没有定义\_\_format\_\_方法, 从object继承的方法会返回str(my_object)

In [56]:
format(v1)

'(3.0, 4.0)'

In [57]:
# 但是这个时候如果传入格式说明就会报错
format(v1, ".2f")

TypeError: unsupported format string passed to Vector2d.__format__

In [58]:
from array import array
import math
# 自己设计向量类v2
class Vector2d(object):
    typecode = "d"
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)  # 实现了iter就会自动拆包
    
    def __str__(self):
        return str(tuple(self)) # 实现了iter方法
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(self)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(self.x or self.y)
    
    @classmethod
    def frombytes(cls, octets):
        cls.typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(cls.typecode)
        return cls(*memv)
    
    def __format__(self, fmt_spec=''):
        componets = (format(c, fmt_spec) for c in self)
        return "({}, {})".format(*componets)

In [59]:
v1 = Vector2d(3,4)
format(v1, ".2f")

'(3.00, 4.00)'

## 9.6 可散列的Vector2d
为了能够散列Vector2d, 就必须实现\_\_hash\_\_方法。

并且可以让x,y属性变为只读

In [64]:
from array import array
import math
# 自己设计向量类v2
class Vector2d(object):
    typecode = "d"
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property    
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)  # 实现了iter就会自动拆包
    
    def __str__(self):
        return str(tuple(self)) # 实现了iter方法
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(self)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(self.x or self.y)
    
    @classmethod
    def frombytes(cls, octets):
        cls.typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(cls.typecode)
        return cls(*memv)
    
    def __format__(self, fmt_spec=''):
        componets = (format(c, fmt_spec) for c in self)
        return "({}, {})".format(*componets)
    
    def __hash__(self):  # 只有不可变的属性才能hash
        return hash(self.x) ^ hash(self.y)

In [65]:
v1 = Vector2d(3, 4)

In [66]:
v1

Vector2d(3.0, 4.0)

In [67]:
hash(v1)

7

## 9.7 Python的私有属性和“受保护的”属性
Python的单下划线为保护变量机制是程序员约定俗成的, Python解释器并不会做什么保护机制。

In [68]:
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

## 9.8 使用\_\_slots\_\_类属性节省空间
默认情况下, Python的各个实例中名为\_\_dict\_\_的字典储存实例属性以提升访问速度。但是为了结省内存, 我们可以使用\_\_slots\_\_类属性,

用元组去保存实例属性，而不用字典。

In [69]:
class Vector2d:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [82]:
Vector2d.__dict__  # 注意这里是类的dict

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Vector2d.__init__(self, x, y)>,
              '__dict__': <attribute '__dict__' of 'Vector2d' objects>,
              '__weakref__': <attribute '__weakref__' of 'Vector2d' objects>,
              '__doc__': None})

In [77]:
v = Vector2d(3, 4)
v.__dict__  # 这是实例的dict

{'x': 3, 'y': 4}

In [104]:
# 使用slots
class Vector2d:
    __slots__ = ("__x", "__y")  # 告诉解释器, 这个类的所有实例属性都在这里
    
    def __init__(self, x, y):
        self.__x = x
        self.__y = y
    
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y

In [105]:
v = Vector2d(3, 4)

In [106]:
v.__dict__  # 这时候就没有__dict__

AttributeError: 'Vector2d' object has no attribute '__dict__'

In [107]:
dir(v)

['_Vector2d__x',
 '_Vector2d__y',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 'x',
 'y']

In [118]:
class A:
    __slots__= ("ax", "ay")
    
    def __init__(self, ax, ay):
        self.ax = ax
        self.ay = ay
        
class B(A):
    __slots__= ("bx", "by")
    
    def __init__(self, bx, by):
        super().__init__(bx, by)
        self.bx = bx
        self.by = by 


In [121]:
x = B(1, 2)
x.ax, x.ay

(1, 2)

In [123]:
x.bx, x.by

(1, 2)

如果用了slots还要用弱引用就需要把weakref添加到slots中

slots需要注意的点：
* 每个子类都需要定义slots, 因为解释器会忽略继承的slots
* 实例只能拥有slots中列出的属性
* 要想用弱引用就需要把weakref添加到slots中

## 9.9 覆盖类属性

In [124]:
# 使用slots
class Vector2d:
    typecode = "d"
    
    def __init__(self, x, y):
        self.__x = x
        self.__y = y
    
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y

In [125]:
v = Vector2d(3, 4)
v.typecode = "f"

In [126]:
Vector2d.typecode  #不会被改变

'd'

In [127]:
v.typecode

'f'